In [1]:
import numpy as np
import pandas as pd

data = pd.read_excel('data_v7.xlsx')


data.drop(['Unnamed: 0','locations_month','label','date'],axis=1,inplace=True)

data.drop(['pressureInches','visibility'],axis=1,inplace=True)


loc_lst = list(data['locations'].unique())

i = 0
for haha in loc_lst:
    
    data.loc[data.locations == haha, 'locations'] = i
    i+=1


from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

scaler = MinMaxScaler(feature_range=[0, 1])
data_rescaled = scaler.fit_transform(data)


# 8 features

pca = PCA(n_components=8)
data_pca = pca.fit_transform(data_rescaled)

In [3]:
labels = pd.read_excel('data_v7.xlsx',usecols=['label'])

data = pd.DataFrame(data_pca,columns=['c1','c2','c3','c4','c5','c6','c7','c8'])

In [4]:
data.head()

,c1,c2,c3,c4,c5,c6,c7,c8
0,0.966233,0.531514,-0.053925,0.033829,0.016468,-0.033678,-0.006474,-0.073370
1,0.742890,0.522499,-0.045905,-0.057583,0.073825,-0.044661,-0.005593,0.003154
2,0.182807,0.484951,0.331263,0.115176,0.118272,-0.002310,0.041395,0.004899
3,-0.351475,0.496609,0.182283,0.062822,-0.059803,-0.066623,0.111783,0.041275
4,-0.709062,0.487533,0.298583,-0.042041,-0.022723,-0.001474,0.064748,-0.070292


In [5]:
X = data.copy()
y = labels.copy()

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [8]:
import tensorflow as tf

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))

In [10]:
train_dataset = dataset.shuffle(len(X_train)).batch(1)

In [11]:
def get_compiled_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

In [12]:
model = get_compiled_model()
model.fit(train_dataset, epochs=10)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/10
1716/1716 [==============================] - 5s 3ms/step - loss: 0.3747 - accuracy: 0.8899
Epoch 2/10
1716/1716 [==============================] - 1s 867us/step - loss: 0.1795 - accuracy: 0.9441
Epoch 3/10
1716/1716 [==============================] - 2s 882us/step - loss: 0.1686 - accuracy: 0.9522
Epoch 4/10
1716/1716 [==============================] - 2s 891us/step - loss: 0.1660 - accuracy: 0.9534
Epoch 5/10
1716/1716 [==============================] - 2s 881us/step - loss: 0.1649 - accurac

In [55]:
nn_pred_test = model.predict(X_test)

In [56]:
nn_pred_train = model.predict(X_train)

In [57]:
y_test.index = [i for i in range(len(y_test))]

In [58]:
y_train.index = [i for i in range(len(y_train))]

In [59]:
np.mean(nn_pred_test[list(y_test[y_test['label'] ==0].index)])

0.051158346

In [62]:
np.mean(nn_pred_train[list(y_train[y_train['label'] ==1].index)])

0.5054629

In [ ]:
y_test = list(y_test['label'])

In [81]:
test_acc = [1 if nn_pred_test[i] > 0.3 else 0 for i in range(len(nn_pred_test))]

In [84]:
train_acc = [1 if nn_pred_train[i] > 0.3 else 0 for i in range(len(nn_pred_train))]

In [77]:
from sklearn.metrics import classification_report,confusion_matrix

In [83]:
print(classification_report(test_acc,y_test))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       791
           1       0.55      0.67      0.61        55

    accuracy                           0.94       846
   macro avg       0.76      0.82      0.79       846
weighted avg       0.95      0.94      0.95       846



In [85]:
print(classification_report(train_acc,y_train))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1608
           1       0.59      0.71      0.65       108

    accuracy                           0.95      1716
   macro avg       0.79      0.84      0.81      1716
weighted avg       0.96      0.95      0.95      1716



In [86]:
print(confusion_matrix(train_acc,y_train))

[[1555   53]
 [  31   77]]


In [88]:
print(confusion_matrix(test_acc,y_test))

[[761  30]
 [ 18  37]]
